In [2]:
from zeep import Client, Settings, helpers
from zeep.plugins import HistoryPlugin
from zeep.transports import Transport
from zeep.exceptions import Fault, TransportError
import requests
import requests_cache
from lxml import etree

session   = requests.Session()
transport = Transport(session=session, timeout=20)
settings  = Settings(strict=False, xml_huge_tree=True)
history   = HistoryPlugin()

WSDL = "https://www.dneonline.com/calculator.asmx?WSDL"

In [3]:
WSDL

'https://www.dneonline.com/calculator.asmx?WSDL'

In [4]:
client = None
MODE = None

def build_client_from(url: str):
    return Client(wsdl=url, settings=settings, transport=transport, plugins=[history])

try:
    session.verify = True
    client = build_client_from(WSDL)
    MODE = "https"
except Exception:
    try:
        session.verify = False
        client = build_client_from(WSDL)
        MODE = "https_verify_false"
    except Exception:
        try:
            http_wsdl = WSDL.replace("https://", "http://")
            print("http is replaced!")
            session.verify = True
            client = build_client_from(http_wsdl)
            MODE = "http"
        except Exception:
            client = None
            MODE = None
            print("Client is none!")

http is replaced!


In [5]:
http_wsdl

'http://www.dneonline.com/calculator.asmx?WSDL'

In [6]:
wsdl = f"Active ({MODE})"
client.wsdl.services.values()

odict_values([<Service(name='Calculator', ports=OrderedDict([('CalculatorSoap', <Port(name='CalculatorSoap', binding=<Soap11Binding(name='{http://tempuri.org/}CalculatorSoap', port_type=<PortType(name='{http://tempuri.org/}CalculatorSoap')>)>, {'address': 'http://www.dneonline.com/calculator.asmx'})>), ('CalculatorSoap12', <Port(name='CalculatorSoap12', binding=<Soap12Binding(name='{http://tempuri.org/}CalculatorSoap12', port_type=<PortType(name='{http://tempuri.org/}CalculatorSoap')>)>, {'address': 'http://www.dneonline.com/calculator.asmx'})>)]))>])

In [7]:
wsdl

'Active (http)'

In [8]:
def list_ops_robust(c, label):
    print(f"-- {label} --")
    for service in c.wsdl.services.values():
        print("Service:", service.name)
        for port in service.ports.values():
            print("  Port:", port.name)
            ops = getattr(port.binding, "_operations", {})
            if isinstance(ops, dict) and ops:
                for name in sorted(ops.keys()):
                    print("    Operation:", name)
            else:
                proxy = c.bind(service.name, port.name)
                names = sorted(n for n in dir(proxy) if not n.startswith("_") and callable(getattr(proxy, n, None)))
                for n in names:
                    print("    Operation:", n)


In [9]:
list_ops_robust(client, wsdl)

-- Active (http) --
Service: Calculator
  Port: CalculatorSoap
    Operation: Add
    Operation: Divide
    Operation: Multiply
    Operation: Subtract
  Port: CalculatorSoap12
    Operation: Add
    Operation: Divide
    Operation: Multiply
    Operation: Subtract


In [10]:
def safe_call(op, **kwargs):
    try:
        return getattr(client.service, op)(**kwargs)
    except Fault as f:
        print(f"{op} → SOAP Fault:", f)
    except TransportError as te:
        print(f"{op} → Transport error:", te)
    except Exception as e:
        print(f"{op} → Error:", type(e).__name__, e)

In [11]:
safe_call("Subtract", intA=360, intB = 2400)

-2040

In [12]:
def show_last_exchange(hist):
    try:
        sent = etree.tostring(hist.last_sent["envelope"], pretty_print=True).decode("utf-8")
        recv = etree.tostring(hist.last_received["envelope"], pretty_print=True).decode("utf-8")
        print("----- SOAP REQUEST -----\n", sent)
        print("\n----- SOAP RESPONSE -----\n", recv)
    except Exception as e:
        print("No captured exchange yet.", e)


In [13]:
show_last_exchange(history)

----- SOAP REQUEST -----
 <soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
  <soap-env:Body>
    <ns0:Subtract xmlns:ns0="http://tempuri.org/">
      <ns0:intA>360</ns0:intA>
      <ns0:intB>2400</ns0:intB>
    </ns0:Subtract>
  </soap-env:Body>
</soap-env:Envelope>


----- SOAP RESPONSE -----
 <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">
  <soap:Body>
    <SubtractResponse xmlns="http://tempuri.org/">
      <SubtractResult>-2040</SubtractResult>
    </SubtractResponse>
  </soap:Body>
</soap:Envelope>



In [14]:
safe_call("Divide", intA=100, intB=10)

10

In [15]:
show_last_exchange(history)

----- SOAP REQUEST -----
 <soap-env:Envelope xmlns:soap-env="http://schemas.xmlsoap.org/soap/envelope/">
  <soap-env:Body>
    <ns0:Divide xmlns:ns0="http://tempuri.org/">
      <ns0:intA>100</ns0:intA>
      <ns0:intB>10</ns0:intB>
    </ns0:Divide>
  </soap-env:Body>
</soap-env:Envelope>


----- SOAP RESPONSE -----
 <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">
  <soap:Body>
    <DivideResponse xmlns="http://tempuri.org/">
      <DivideResult>10</DivideResult>
    </DivideResponse>
  </soap:Body>
</soap:Envelope>



In [16]:
def circumference(radius:float) -> float:
    if radius < 0:
        raise ValueError ("Radius must be positive!")
    first_one = safe_call("Multiply", intA = 2, intB = 3)
    second_one = safe_call("Multiply", intA = first_one, intB = radius)
    return second_one

In [17]:
for r in (0, 1, 2, 500):
    print(f"r={r:>4}: C= {circumference(r):>4} vs {3.14 * 2 * r}")

r=   0: C=    0 vs 0.0
r=   1: C=    6 vs 6.28
r=   2: C=   12 vs 12.56
r= 500: C= 3000 vs 3140.0
